# IMPORTS

In [2]:
#import necessary dependecies
import os
import warnings
import numpy as np  
import pandas as pd, os, gc , random
from tqdm import tqdm_notebook as tqdm 
warnings.filterwarnings('ignore') 
np.random.seed(111)
random.seed(111)
from sklearn.model_selection import  StratifiedKFold

In [3]:
import datetime
import pandas as pd
from time import time
# from autograd import grad
# import autograd.numpy as np
import numpy as np
from numba import njit
from scipy.optimize import minimize, fsolve
from sklearn.metrics import log_loss

# Get Data

In [4]:
Train = pd.read_csv('Train.csv')
Train.head()

,img_IDS,Label
0,ImageID_33HMDMJ3,Temple
1,ImageID_V5POEX4O,Church
2,ImageID_89CCCCW6,Enough/Satisfied
3,ImageID_GNJXOWX5,Me
4,ImageID_7Q9LOP7R,Love


In [5]:
targets = Train.Label.unique().tolist()

Target_Mapper = dict(zip(targets,[i for i in range(len(targets))]))
InverseTarget_Mapper = dict(zip([i for i in range(len(targets))],targets))

In [6]:
Train.Label = Train.Label.map(Target_Mapper)

In [7]:
Test = pd.read_csv('Test.csv')

# ENSEMBLE

In [ ]:
test_dict = {
            'SwinLarge384': 'AssazzinBaseline_swin_large_patch4_window12_384_GmeanPreds_TTAx4.npy', 
            'SwinBase384': 'AssazzinBaseline_swin_base_patch4_window12_384_GmeanPreds_TTAx4.npy', 
            'ConvnextXLarge384': 'AssazzinBaseline_convnext_xlarge_GmeanPreds_TTAx4.npy' ,
            'ConvnextLarge384': 'AssazzinBaseline_convnext_large_GmeanPreds_TTAx4.npy' ,
            'ConvnextBase384': 'AssazzinBaseline_convnext_base_GmeanPreds_TTAx4.npy' ,
            'tf_efficientnet_l2_ns_475': 'AssazzinBaseline_tf_efficientnet_l2_ns_475_GmeanPreds_TTAx4.npy' ,
            'tf_efficientnet_b7_ns': 'AssazzinBaseline_tf_efficientnet_b7_ns_GmeanPreds_TTAx4.npy' ,
           }

BlendTest = np.zeros((len(test_dict), Test.shape[0],y_true.shape[1]))
for i in range(BlendTest.shape[0]):
    BlendTest[i] = np.load(list(test_dict.values())[i])

In [ ]:
from scipy.stats.mstats import gmean

In [ ]:
def weighted_gmean(weights,predictions) :
  weighted_gmean = 0
  sum_1 =0
  sum_2 =0
  for i in range(len(weights)) :
    sum_1 += weights[i]* np.log(predictions[i])
    sum_2 += weights[i]
  weighted_gmean += np.exp(sum_1/sum_2)
  return weighted_gmean

In [ ]:
GmeanPreds = weighted_gmean([0.1170513539840638, 0.04436504005465399, 0.11781987384002074,
                             0.16535207293159604, 0.1495350418075401, 0.18654591094796275, 
                             0.2193307064341627],BlendTest)

In [ ]:
sub = pd.DataFrame(GmeanPreds)
sub = sub.rename(columns=InverseTarget_Mapper)
sub['img_IDS'] = Test['img_IDS'].values

submission = pd.read_csv('SampleSubmission.csv')
sub = pd.merge(sub,submission[['img_IDS']],on='img_IDS',how='right')
for col in submission.columns[1:] :
  submission[col] = sub[col]

In [ ]:
submission.head()

,img_IDS,Church,Enough/Satisfied,Friend,Love,Me,Mosque,Seat,Temple,You
0,ImageID_USRB8QNG,0.001711,0.000192,0.000062,0.000086,0.000114,0.000185,0.000043,0.995613,0.000294
1,ImageID_SZ8D1ZJI,0.000550,0.994667,0.000309,0.000034,0.000086,0.000223,0.000033,0.000158,0.000148
2,ImageID_4OJO2F8J,0.022211,0.002093,0.000888,0.001297,0.000688,0.969334,0.000698,0.001071,0.000530
3,ImageID_IEE4XV0B,0.014623,0.001869,0.000712,0.000611,0.000860,0.974974,0.002076,0.000997,0.000712
4,ImageID_BP0O0WZ9,0.000122,0.000270,0.000062,0.000062,0.000626,0.000098,0.000766,0.000702,0.986968


In [ ]:
submission.to_csv(f'ASSAZZIN_WINNING_SUBMISSION.csv',index=False)